# Chains

Chains allow us to link the output of one LLM call as the input of another call.

Topics:
- LLMChain
- SimpleSequentialChain
- SequentialChain
- LLMRouterChain
- TransformChain
- OpenAI Function Calling
- MathChain
- AdditionalChains

# Chain object

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

In [2]:
human_prompt = HumanMessagePromptTemplate.from_template("Make up a funny company name for a company that makes: {product}")

chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [3]:
chat = ChatOpenAI()

In [4]:
from langchain.chains import LLMChain

In [5]:
# 2 params: LLM and Prompt
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [6]:
result = chain.run(product="Computers")

In [7]:
result

'Byte Me Computers'

Output of chains is not as complex as each individual object.

# SimpleSequentialChain

We now know how to create a single LLMChain, and we can now combine multiple chains.

input --> LLMChain --> LLMChain --> LLMChain --> Output

The chain of chains is a SimpleSequentialChain

In [8]:
from langchain.chains import LLMChain, SimpleSequentialChain

In [9]:
llm = ChatOpenAI()

In [10]:
# Topic for a blog post --> [[ Outline --> Create blog post from outline ]] --> Blog post text

Usually, multiple chains share the same model. But you can use different models. For example, a cheap model for a simpler task and a heavy model for a complex one.

In [11]:
# First chain
template = "Give me a simple bullet point outline for a blog post on {topic}"
first_prompt = ChatPromptTemplate.from_template(template)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [12]:
# Second chain
template2 = "Write a blog post using this outline {outline}"
second_prompt = ChatPromptTemplate.from_template(template2)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [13]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

How does the chain know the variable names? (Topic and Outline). This is a limitation of the SimpleSequentialChain. It is limited to one input and one output. With a SequentialChain, we have that flexibility.

In [23]:
result = full_chain.run("Cheesecake")



> Entering new SimpleSequentialChain chain...
- Introduction to cheesecake as a popular dessert
- Brief history of cheesecake's origins and evolution
- Different types of cheesecake such as classic, New York-style, and specialty flavors
- Ingredients commonly used in cheesecake recipes
- Step-by-step instructions for making a basic cheesecake from scratch
- Tips and tricks for achieving the perfect texture and flavor
- Creative variations and toppings to enhance your cheesecake
- Ideas for serving and presenting cheesecake
- Healthier alternatives and substitutions for those with dietary restrictions
- Conclusion and final thoughts on the versatility and deliciousness of cheesecake
Introduction to Cheesecake as a Popular Dessert

Cheesecake is a beloved dessert that has captured the hearts (and taste buds) of people around the world. With its rich and creamy texture, combined with a sweet and tangy flavor, cheesecake has become a staple in many dessert menus. In this blog post, we wi

In [25]:
print(result)

Introduction to Cheesecake as a Popular Dessert

Cheesecake is a beloved dessert that has captured the hearts (and taste buds) of people around the world. With its rich and creamy texture, combined with a sweet and tangy flavor, cheesecake has become a staple in many dessert menus. In this blog post, we will explore the origins and evolution of cheesecake, the different types and flavors available, as well as tips and tricks for making the perfect cheesecake at home.

Brief History of Cheesecake's Origins and Evolution

Cheesecake can trace its roots back to ancient Greece, where it was served to athletes during the first Olympic games in 776 BC. The Greeks believed that cheesecake provided energy and strength to the athletes. Over time, the recipe for cheesecake spread throughout Europe, with each region adding its own unique twist. In the United States, cheesecake gained popularity in the 19th century and has since become a classic American dessert.

Different Types of Cheesecake

Th

# SequentialChain

Very similar to SimpleSequentialChains, but allow to have access to all outputs from the internal LLMChains.

In [14]:
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [15]:
llm = ChatOpenAI()

In [16]:
# 3 Chains
# Employee performance review input text
# review_text --> LLMChain --> summary
# Summary --> LLMChain --> Weaknesses
# Weaknesses --> LLMChain --> Improvement Plan

In [34]:
template1 = "Give a summary of this employee's performance review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="review_summary")  # specify output so next chain knows

In [35]:
template2 = "Identify key employee weaknesses from this performance review summary:\n{review_summary}"  # needs to match output key
prompt2 = ChatPromptTemplate.from_template(template2)
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="weaknesses")

In [38]:
template3 = "Create a personalised improvement plan to help address and fix these weaknesses:\n{weaknesses}"  # needs to match output key
prompt3 = ChatPromptTemplate.from_template(template3)
chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="final_plan")

In [40]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [41]:
full_chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["review"],  # include the input of first chain
    output_variables=["review_summary", "weaknesses", "final_plan"],  # good practice to include all
    verbose=True
)

In [43]:
result = full_chain(employee_review)  # no run.()



> Entering new SequentialChain chain...

> Finished chain.


Verbose here does not do anything because we have access to all results in output. A dict with each step.

In [45]:
type(result)

dict

In [46]:
result.keys()

dict_keys(['review', 'review_summary', 'weaknesses', 'final_plan'])

In [53]:
print(result["review_summary"])

Summary:

Joe Schmo, a software engineer, has received a positive performance review. His strengths include his technical expertise, collaborative nature, initiative and self-motivation, adaptability, and exceptional problem-solving skills. He is highly skilled in programming languages and software development best practices. Joe actively engages with cross-functional teams and seeks input from others. He consistently demonstrates initiative and takes the lead in seeking out new projects. His adaptability allows him to handle changing project requirements and learn new technologies. Joe approaches issues with a logical mindset and finds effective solutions. 

However, there are areas for improvement. Joe occasionally struggles with time management, leading to missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would enhance his efficiency. His written communication skills could also be improved t

In [52]:
print(result["weaknesses"])

Key employee weaknesses identified in the performance review summary for Joe Schmo, the software engineer, include:

1. Time management: Joe occasionally struggles with time management, leading to missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would enhance his efficiency.

2. Written communication skills: Joe's written communication skills could be improved to ensure clarity in documentation. This weakness suggests a need for clearer and more effective written communication in his work.

3. Reluctance to delegate tasks: Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging appropriate delegation will create a more balanced and productive team environment.


In [51]:
print(result["final_plan"])

Personalised Improvement Plan for Joe Schmo:

1. Time Management:
- Set clear goals and deadlines for each task or project.
- Prioritize tasks based on their importance and urgency.
- Break down larger tasks into smaller, manageable steps.
- Use time management tools such as calendars or task management apps.
- Regularly review and adjust the schedule as needed.
- Seek guidance or training on time management techniques if necessary.

2. Written Communication Skills:
- Take the time to carefully proofread and edit written communication before submitting or sharing.
- Use clear and concise language to convey ideas.
- Seek feedback from colleagues or supervisors on written communication to identify areas for improvement.
- Consider taking a business writing course or workshop to enhance written communication skills.
- Practice writing regularly, both professionally and personally.

3. Delegation:
- Assess tasks to determine which ones can be delegated to others.
- Identify team members wi

# LLMRouterChain
- Takes an input and redirects it to the most appropriate LLMChain sequence
- The router accepts multiple potential destination LLMChains and then via a specialized prompt, the router will reat the initial input then output a spefic dictionary that matches up to one of the potential destination chains to continue processing.

The most important things to build are the:
- Templates
- Prompt infos dictionary

The rest is boilerplate LangChain code.

In [26]:
# Example: Input --> LLMRouter  --> Path 1: LLMCHain of customer support
#                               --> Path 2: LLMChain of internal employee

In [27]:
# student ask question about Physics
# "How does a magnet work?"
# "Explain what is a Feynman diagram?"
# Questions that can be really simple or difficult
# Input --> Router --> LLM decides the correct chain --> Chain --> Output

In [28]:
from langchain.chains import RouterChain

In [29]:
beginer_template = """You are a physics teacher who is really focused on beginners and explaining complex concepts in simple to understand terms.
You assume no prior knowledge. Here is your question:\n{input}"""

In [30]:
expert_template = """You are a physics professor who explains physics topics to advanced audience members. 
You can assume anyone you answer has a PhD in Physics. Here is your question:\n{input}"""

In [31]:
# Route prompt information
# [{Name, Description, Template}]

prompt_infos = [
    {
        "name": "Beginner Physics",
        "description": "Answers basic physics questions",  # Important that this matches the template phrasing.
        "template": beginer_template,
    },
    {
        "name": "Expert Physics",
        "description": "Answers advanced physics questions",  # Important that this matches the template phrasing.
        "template": expert_template,
    }
]

In [32]:
llm = ChatOpenAI()

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    template = p_info["template"]

    prompt = ChatPromptTemplate.from_template(template=template)

    chain = LLMChain(llm=llm, prompt=prompt)

    destination_chains[name] = chain
    

In [55]:
destination_chains.keys()

dict_keys(['Beginner Physics', 'Expert Physics'])

In [33]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [34]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [35]:
# This is just a set of instructions
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [36]:
# Edit the MULTI PROMPT ROUTER TEMPLATE
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations

['Beginner Physics: Answers basic physics questions',
 'Expert Physics: Answers advanced physics questions']

In [37]:
destination_str = "\n".join(destinations)  # we should follow this format
destination_str

'Beginner Physics: Answers basic physics questions\nExpert Physics: Answers advanced physics questions'

In [38]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [39]:
# Destinations are now filled
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destination_str
)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Beginner Physics: Answers basic physics questions
Expert Physics: Answers advanced physics questions


In [48]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

In [49]:
from langchain.chains.router import MultiPromptChain

In [50]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [51]:
chain = MultiPromptChain(
    router_chain=router_chain, 
    destination_chains=destination_chains, 
    default_chain=default_chain,
    verbose=True
)

In [52]:
import warnings
warnings.filterwarnings("ignore")

result = chain.run("How do magnets work?")



> Entering new MultiPromptChain chain...
Beginner Physics: {'input': 'How do magnets work?'}
> Finished chain.


In [53]:
print(result)

Great question! Let's start by understanding what magnets are. Magnets are special materials that have the ability to attract certain objects made of metal, like iron or steel. They can also repel or push away other magnets. 

Now, magnets work because of something called magnetic fields. Imagine that magnets have invisible lines of force around them, like a force field. These lines of force create a magnetic field that surrounds the magnet. 

When two magnets are brought close together, their magnetic fields interact with each other. There are two important things to remember about these interactions:

1. Like poles repel: Magnets have two ends, called poles, known as the North pole (N) and South pole (S). The North pole of one magnet will repel the North pole of another magnet, and the same goes for the South poles. So, if you try to bring two North poles together, they will push each other away. The same happens with two South poles.

2. Opposite poles attract: On the other hand, if

In [54]:
result = chain.run("Please explain Feynman diagrams")
print(result)



> Entering new MultiPromptChain chain...
Expert Physics: {'input': 'Please explain Feynman diagrams'}
> Finished chain.
Feynman diagrams are powerful tools used in theoretical physics, particularly in the field of quantum field theory, to visualize and calculate the interactions between elementary particles. They were introduced by physicist Richard Feynman in the 1940s and have since become an integral part of particle physics.

At its core, a Feynman diagram represents a specific mathematical expression called a Feynman amplitude, which describes the probability amplitude for a particular particle interaction to occur. Feynman amplitudes are derived from the principles of quantum mechanics and allow us to calculate the likelihood of various particle interactions.

In a Feynman diagram, particles are represented by lines, and the interactions between them are depicted as vertices. The lines can be straight or wavy, depending on the nature of the particle being represented. For examp

# Transform Chain

In [43]:
yelp_review = open("yelp_review.txt").read()

Instead of asking LLM to lower the text and some other cleaning (more expensive), we can do it using a transform chain.

In [45]:
print(yelp_review)

TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KN

In [48]:
# We want something like this: Lowercase and only the review text
print(yelp_review.split("REVIEW:")[-1].lower())


oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowledgeable about the menu. they went above and beyond

In [49]:
# Input --> custom python transformation --> LLMChain

In [50]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [62]:
def transformer_fun(inputs: dict) -> dict:
    text = inputs["text"]
    only_review_text = text.split("REVIEW")[-1]
    lower_case_text = only_review_text.lower()
    return {"output": lower_case_text}

In [63]:
transform_chain = TransformChain(
    input_variables=["text"],
    output_variables=["output"],
    transform=transformer_fun
)

In [64]:
template = "Create a one sentence summary of this review:\n{review}"

In [65]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(template)
summary_chain = LLMChain(llm=llm, prompt=prompt, output_key="review_summary")

In [66]:
sequential_chain = SimpleSequentialChain(
    chains=[transform_chain, summary_chain],
    verbose=True
)

In [67]:
result = sequential_chain(yelp_review)



> Entering new SimpleSequentialChain chain...
:
oh my goodness, where do i begin? this restaurant is absolutely phenomenal! i went there last night with my friends, and we were blown away by the experience!

first of all, the ambiance is out of this world! the moment you step inside, you're greeted with a warm and inviting atmosphere. the decor is stunning, and it immediately sets the tone for an unforgettable dining experience.

now, let's talk about the food! wow, just wow! the menu is a paradise for food lovers. every dish we ordered was a masterpiece. the flavors were bold, vibrant, and exploded in our mouths. from starters to desserts, every bite was pure bliss!

their seafood platter is a must-try! the freshness of the seafood is unmatched, and the presentation is simply stunning. i have never tasted such delicious and perfectly cooked seafood in my life. it's a seafood lover's dream come true!

the service was exemplary. the staff was attentive, friendly, and extremely knowled

In [69]:
print(result["output"])

This review is a glowing endorsement of a restaurant with phenomenal ambiance, delicious food, impeccable service, and heavenly desserts, making it a hidden gem for a memorable dining experience.


# OpenAI function calling with LangChain

In [70]:
from langchain.chat_models import ChatOpenAI

In [79]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [92]:
# Not using pydantic
class Scientist():
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

In [82]:
json_schema = {
    "title": "Scientist",
    "description": "Information about a famous scientist",
    "type": "object",
    "properties": {
        "first_name": {
            "title": "First Name",
            "description": "First name of scientist",
            "type": "string"
        },
        "last_name": {
            "title": "Last Name",
            "description": "Last name of scientist",
            "type": "string"
        }
    },
    "required": ["first_name", "last_name"]
}

In [85]:
template = "Name a famous {country} scientist"

In [87]:
from langchain.chains.openai_functions import create_structured_output_chain

chat_prompt = ChatPromptTemplate.from_template(template)

In [88]:
chain = create_structured_output_chain(json_schema, llm, chat_prompt, verbose=True)

In [89]:
result = chain.run("American")



> Entering new LLMChain chain...
Prompt after formatting:
Human: Name a famous American scientist

> Finished chain.


albert = Scientist(result["first_name"], result["last_name"])

In [95]:
albert.first_name

'Albert'

# Math Chain

In [97]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage

In [98]:
model = ChatOpenAI()
result = model([HumanMessage(content="What is 2 + 2")])

In [99]:
result.content

'2 + 2 equals 4.'

In [100]:
17**11

34271896307633

With mathematical operations, the model starts to hallucinate.

In [101]:
result = model([HumanMessage(content="What is 17 raised to the power of 11")])
print(result.content)

17 raised to the power of 11 is equal to 137,858,491,849.


In [102]:
eval("17**11")

34271896307633

In [103]:
from langchain import LLMMathChain

In [104]:
llm_math_model = LLMMathChain.from_llm(model)

In [107]:
llm_math_model("What is 17 raised to the power of 11")

{'question': 'What is 17 raised to the power of 11',
 'answer': 'Answer: 34271896307633'}

# QA Documents

In [108]:
# Vector store
# QA on that vector store

In [110]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [130]:
embedding_function = OpenAIEmbeddings()
db_connection = Chroma(persist_directory="US_Constitution", embedding_function=embedding_function)

In [131]:
from langchain.chains.question_answering import load_qa_chain

In [132]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain # very similar, more metadata

In [133]:
from langchain.chat_models import ChatOpenAI

In [134]:
llm = ChatOpenAI(temperature=0)

In [135]:
chain = load_qa_chain(
    llm=llm,
    chain_type="stuff"  # means we are going to insert ("stuff") some context from vector store
)

In [136]:
question = "What is the 15th amendment?"

In [140]:
docs = db_connection.similarity_search(question)  # Document that will be context

In [141]:
chain = load_qa_with_sources_chain(llm, chain_type="stuff")
chain.run(
    input_documents=docs,
    question=question
)

'The 15th Amendment to the United States Constitution prohibits the denial or abridgment of the right to vote based on race, color, or previous condition of servitude. It grants Congress the power to enforce this amendment through appropriate legislation.\nSOURCES: some_data/US_Constitution.txt'